<a href="https://colab.research.google.com/github/DorShabat/Cryptology-Project/blob/main/Schnorr_signature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Schnorr signature

by https://www.youtube.com/watch?v=mV9hXEFUB6A&ab_channel=Theoretically

## pips:

In [2]:
%%capture
!pip install cryptography

##imports:

In [1]:
import numpy as np
import sympy
import os
import random
import hashlib
from sympy.ntheory.generate import randprime

## functions for generating the mathematical parameters:

In [2]:
def get_biggest_factor_of_prime_minus_one(prime):
    if not sympy.isprime(prime): # Check if the input is a prime number
        raise ValueError("The input number is not a prime number.")
    p_minus_1 = prime - 1
    factors = sympy.factorint(p_minus_1) # Find a factor of P - 1
    return max(factors.keys()) # Return biggest factor of P - 1


def mod_exp(base, exp, mod):
    """
    Function to perform modular exponentiation.
    It returns (base^exp) % mod.
    """
    result = 1
    base = base % mod
    while exp > 0:
        if exp % 2 == 1:  # If exp is odd, multiply base with result
            result = (result * base) % mod
        exp = exp >> 1    # exp = exp // 2
        base = (base * base) % mod  # base = base^2
    return result

def find_a(Q, P):
    """
    Function to find a non-trivial A such that A^Q ≡ 1 (mod P).
    """
    if Q == 0:
        raise ValueError("Q must be non-zero.")
    if P <= 1:
        raise ValueError("P must be greater than 1.")

    A = 2
    while A < P:
        if mod_exp(A, Q, P) == 1:
            return A
        A += 1

    raise ValueError("No non-trivial solution found.")

def mod_inverse(A, Q):
    """
    Function to find the modular inverse of A under modulo Q using the Extended Euclidean Algorithm.
    """
    def extended_gcd(a, b):
        if a == 0:
            return b, 0, 1
        gcd, x1, y1 = extended_gcd(b % a, a)
        x = y1 - (b // a) * x1
        y = x1
        return gcd, x, y

    gcd, x, y = extended_gcd(A, Q)
    if gcd != 1:
        raise ValueError(f"No modular inverse exists for A = {A} and Q = {Q}")
    else:
        return x % Q

def compute_V(A, s, Q):
    """
    Function to compute V such that V = (A^(-s)) mod Q.
    """
    A_inv = mod_inverse(A, Q)
    V = mod_exp(A_inv, s, Q)
    return V

def hash_function(data):
    """
    Function to perform SHA-256 hashing.
        Parameters:
    data (str): The input data to hash.
        Returns:
    str: The hexadecimal representation of the SHA-256 hash of the input data.
    """
    # Ensure the input data is in bytes
    if isinstance(data, str):
        data = data.encode('utf-8')
    # Create a new SHA-256 hash object
    sha256 = hashlib.sha256()
    # Update the hash object with the data
    sha256.update(data)
    # Return the hexadecimal representation of the hash
    return sha256.hexdigest()

##Networking:

# Alice

***Sign:***

`M = message to be signed`

global:
```
P = a prime number     # typically 1024-bit
Q = a factor of P-1    # typically 160-bit
A = a^Q === 1 mod P
```
private key:
`s = random 0 < s < Q`

public key:
`V = a^(-s) mod Q`

for signing:
```
r = random 0 < r < Q
x = A^r mod P
e = Hash(M||x)
y = (r+se)modQ
```

Send: ` Message, M | Signature(e , y) `


### generate parameters:

In [3]:
M = b'This is a test message with my credit card digits.' # binary msg

P = randprime(10000000, 20000000) # rand a prime number.
Q = get_biggest_factor_of_prime_minus_one(P)
A = find_a(Q, P)
s = random.randint(0, Q)
V = compute_V(A, s, Q)
r = random.randint(0, Q)
x = mod_exp(A, r, P)
e = hash_function(str(M) + str(x))[:10]
y = (r + s*int(e, 16)) % Q

### hard coded:

In [20]:
M = b'This is a test message with my credit card digits.' # binary msg

P = 17565463
Q = 51361
A = 169
s = 37613
V = 7299
r = 41044
x = 1215921
e = hash_function(str(M) + str(x))[:50]
y = 26752

##sending to Bob......

### parameters printing:

In [21]:
print(f'P = {P}')
print(f'Q = {Q}')
print(f'A = {A}')
print(f's = {s}')
print(f'V = {V}')
print(f'r = {r}')
print(f'x = {x}')
print(f'e = {e}')
print(f'y = {y}')

P = 17565463
Q = 51361
A = 169
s = 37613
V = 7299
r = 41044
x = 1215921
e = 545ff80658331b4c4043ed4ffdec25b0fb3cf17e7308e11704
y = 26752


# Bob


*** Verification:***
  
Received:
`(M,  e,  y)`

Known publicly:
`(A,  P,  Q,  V)`

compute: `x' = (A^y * V^e) mod P`

### print what recived and known:

In [22]:
print("   Recived form alice:\n")
print(f'M = {M}')
print(f'e = {e}')
print(f'y = {y}')
print("\n\n  known public parameters:\n")
print(f'A = {A}')
print(f'P = {P}')
print(f'Q = {Q}')
print(f'V = {V}')

   Recived form alice:

M = b'This is a test message with my credit card digits.'
e = 545ff80658331b4c4043ed4ffdec25b0fb3cf17e7308e11704
y = 26752


  known public parameters:

A = 169
P = 17565463
Q = 51361
V = 7299


### compute:

In [ ]:
## recived msg form alice...

Ay = np.power(A, y)
Ve = np.power(V, int(e, 16))
AyVe = np.multiply(Ay, Ve)

x_tag = np.mod(AyVe, P)

print(Ay)
print(Ve)
print(AyVe)
print(f'\nx\'= {x_tag}')

print(f'x = {x}\n')


e_tag = hash_function(str(M) + str(x_tag))[:50]
print(f'e\'= {e_tag}')
print(f'e = {e}')

